In [ ]:
# !git init
# !git add .
# !git remote add origin https://github.com/Jonlenes/MscProject.git

!git add .
!git commit -m "Separando em arquivos"
!git push -u origin master

###Imports 

In [2]:
!nvidia-smi

Tue Nov  6 18:58:35 2018       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 396.44                 Driver Version: 396.44                    |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  On   | 00000000:00:1B.0 Off |                    0 |
| N/A   46C    P0    99W / 300W |  15669MiB / 16160MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
|   1  Tesla V100-SXM2...  On   | 00000000:00:1C.0 Off |                    0 |
| N/A   40C    P0    52W / 300W |  15379MiB / 16160MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
|   2  T

In [11]:
import numpy as np
from keras.datasets import mnist, cifar10

from keras.models import Sequential, Model
from keras.layers import Input, Dense, Reshape, Flatten, Dropout, ZeroPadding2D, Activation
from keras.layers import Conv2D, Conv2DTranspose, UpSampling2D
from keras.layers import LeakyReLU, Dropout
from keras.layers import BatchNormalization
from keras.optimizers import Adam, RMSprop

from GANLib.IDataset import IDataset
from GANLib.IDCGANModel import IDCGANModel


In [12]:
class MINISTDataset(IDataset):
    def __init__(self):
        super(MINISTDataset, self).__init__()
    
    def load_data(self):
        print("Dowloand/Extract MNIST")
        (x_train, _), (_, _) = mnist.load_data()
        x_train = x_train / 255.0
        x_train = x_train.reshape(-1, self.height, self.width, self.channel)
        self.x_train = x_train
        print("Finished: ", x_train.shape)

In [13]:
class NoiseDataset(IDataset):
    def __init__(self):
        super(NoiseDataset, self).__init__()
    
    
    def load_data(self):
        pass
        
    
    def __len__(self): 
        return np.int64(99999999999)
        
    
    def nexts(self, lot_size):
        return np.random.uniform(-1.0, 1.0, size=[lot_size, 100])

In [14]:
class DCGANModel(IDCGANModel):
    def _build_new_discriminator(self):
        
        input_shape=(28, 28, 1)
        depth = 64
        dropout = 0.4

        model = Sequential()

        model.add(Conv2D(depth*1, 5, strides=2, input_shape=input_shape, padding='same'))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(dropout))

        model.add(Conv2D(depth*2, 5, strides=2, padding='same'))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(dropout))

        model.add(Conv2D(depth*4, 5, strides=2, padding='same'))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(dropout))

        model.add(Conv2D(depth*8, 5, strides=1, padding='same'))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(dropout))

        # Out: 1-dim probability
        model.add(Flatten())
        model.add(Dense(1))
        model.add(Activation('sigmoid'))
        #model.summary()
        
        return model
      
    
    def _build_new_generator(self):

        dropout = 0.4
        depth = 64+64+64+64
        dim = 7
        # In: 100
        # Out: dim x dim x depth
        model = Sequential()
        
        model.add(Dense(dim*dim*depth, input_dim=100))
        model.add(BatchNormalization(momentum=0.9))
        model.add(Activation('relu'))
        model.add(Reshape((dim, dim, depth)))
        model.add(Dropout(dropout))

        # In: dim x dim x depth
        # Out: 2*dim x 2*dim x depth/2
        model.add(UpSampling2D())
        model.add(Conv2DTranspose(int(depth/2), 5, padding='same'))
        model.add(BatchNormalization(momentum=0.9))
        model.add(Activation('relu'))

        model.add(UpSampling2D())
        model.add(Conv2DTranspose(int(depth/4), 5, padding='same'))
        model.add(BatchNormalization(momentum=0.9))
        model.add(Activation('relu'))

        model.add(Conv2DTranspose(int(depth/8), 5, padding='same'))
        model.add(BatchNormalization(momentum=0.9))
        model.add(Activation('relu'))

        # Out: 28 x 28 x 1 grayscale image [0.0,1.0] per pix
        model.add(Conv2DTranspose(1, 5, padding='same'))
        model.add(Activation('sigmoid'))
        #model.summary()

        return model


In [15]:
model = DCGANModel()

model.compile_discriminator()
model.compile_adversarial()

trainer = GANTrainer(model, MINISTDataset(), NoiseDataset(), save_epoch_interval=10, preview_epoch=True)
trainer.fit(epochs=10)

# model.save_discriminator('result/model_minist_dis_v1.h5')
# model.save_generator('result/model_minist_gen_v1.h5')

InternalError: CUDA runtime implicit initialization on GPU:0 failed. Status: out of memory